In [1]:
from pythainlp import word_tokenize, Tokenizer
from pythainlp.util.trie import Trie
from pythainlp.util import normalize
from pythainlp.corpus import thai_stopwords
from pythainlp.corpus.common import thai_words
from collections import Counter
import numpy as np
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [2]:
stopwords = list(thai_stopwords())

In [3]:
df = pd.read_csv('data/prachatai_test.csv', sep=',')
df.drop('Unnamed: 0',axis='columns', inplace=True)

In [4]:
df.head()

,url,date,title,body_text,politics,human_rights,quality_of_life,international,social,environment,economics,culture,labor,national_security,ict,education
0,https://prachatai.com/print/62490,2015-11-17 18:14,แฮคเกอร์ Anonymous ลั่นทำสงครามไซเบอร์ครั้งใหญ...,17 พ.ย. 2558 Blognone [1] รายงานว่า กลุ่มแฮคเก...,0,0,0,1,0,0,0,0,0,0,1,0
1,https://prachatai.com/print/48181,2013-08-14 20:08,สตูดิโอจิบลิต้านสงคราม วิจารณ์การแก้รัฐธรรมนูญ...,ขณะที่ ส.ส. ญี่ปุ่นต้องการแก้รัฐธรรมนูญเพื่อเป...,0,0,0,1,0,0,0,0,0,0,0,0
2,https://prachatai.com/print/63379,2016-01-09 13:43,We need Safety Zone สมาคมเพื่อสันติภาพนำเดินรณ...,สมาคมเพื่อสันติภาพ นำเครือข่ายเด็กและเยาวชน ภา...,1,0,0,0,1,0,0,0,0,1,0,0
3,https://prachatai.com/print/74297,2017-11-26 09:17,รัตโก มลาดิช ทหารใหญ่กองกำลังชาวเซิร์บถูกตัดสิ...,อดีตเสนาธิการกองทัพเซิร์บบอสเนีย รัตโก มลาดิช ...,0,1,0,1,0,0,0,0,0,0,0,0
4,https://prachatai.com/print/8103,2006-04-10 18:55,พันธมิตรฯใต้เตรียมฟ้อง กกต.เอื้อประโยชน์ให้ไทย...,ประชาไท - มติพันธมิตรฯ ภาคใต้เดินหน้าจัดเวทีปฏ...,1,0,0,0,0,0,0,0,0,0,0,0


In [5]:
def text_preprocess(text):
    text = normalize(text)
    text = re.sub('\s+', ' ', text)
    regex = re.compile(r"[^\u0E00-\u0E7Fa-zA-Z' ]|^'|'$|''")
    text = regex.sub('', text)
    new_words = {"ไม่มี", "ไม่ดี"}
    custom_words_list = set(thai_words())
    custom_words_list.update(new_words)
    trie = Trie(custom_words_list)

    custom_tokenizer = Tokenizer(custom_dict=trie, engine='newmm')
    output = custom_tokenizer.word_tokenize(text)

    symbols = {" ", "  "}
    last_value = [i for i in output if  i not in symbols and len(i) > 2]

    last_list = " ".join(i for i in last_value if i not in stopwords)
    return last_list

In [6]:
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

In [7]:
# df['text_tokens'] = df['body_text'].progress_apply(text_preprocess)

In [8]:
import pickle

# pickle.dump(df, open("data/dataframe2.pkl", "wb"))

In [9]:
df = pickle.load(open("data/dataframe.pkl", "rb"))
df

,url,date,title,body_text,politics,human_rights,quality_of_life,international,social,environment,economics,culture,labor,national_security,ict,education,text_tokens
0,https://prachatai.com/print/62490,2015-11-17 18:14,แฮคเกอร์ Anonymous ลั่นทำสงครามไซเบอร์ครั้งใหญ...,17 พ.ย. 2558 Blognone [1] รายงานว่า กลุ่มแฮคเก...,0,0,0,1,0,0,0,0,0,0,1,0,Blognone รายงาน แฮคเกอร์ Anonymous ประกาศสงครา...
1,https://prachatai.com/print/48181,2013-08-14 20:08,สตูดิโอจิบลิต้านสงคราม วิจารณ์การแก้รัฐธรรมนูญ...,ขณะที่ ส.ส. ญี่ปุ่นต้องการแก้รัฐธรรมนูญเพื่อเป...,0,0,0,1,0,0,0,0,0,0,0,0,ญี่ปุ่น ต้องการ แก้ รัฐธรรมนูญ กองกำลัง ป้องกั...
2,https://prachatai.com/print/63379,2016-01-09 13:43,We need Safety Zone สมาคมเพื่อสันติภาพนำเดินรณ...,สมาคมเพื่อสันติภาพ นำเครือข่ายเด็กและเยาวชน ภา...,1,0,0,0,1,0,0,0,0,1,0,0,สมาคม สันติภาพ เครือข่าย เด็ก เยาวชน ประชา สัง...
3,https://prachatai.com/print/74297,2017-11-26 09:17,รัตโก มลาดิช ทหารใหญ่กองกำลังชาวเซิร์บถูกตัดสิ...,อดีตเสนาธิการกองทัพเซิร์บบอสเนีย รัตโก มลาดิช ...,0,1,0,1,0,0,0,0,0,0,0,0,เสนาธิการ กองทัพ เซิร์บ บอสเนีย รัต ตัดสิน มีค...
4,https://prachatai.com/print/8103,2006-04-10 18:55,พันธมิตรฯใต้เตรียมฟ้อง กกต.เอื้อประโยชน์ให้ไทย...,ประชาไท - มติพันธมิตรฯ ภาคใต้เดินหน้าจัดเวทีปฏ...,1,0,0,0,0,0,0,0,0,0,0,0,ประชา มติ พันธมิตร ภาคใต้ เดินหน้า เวที ปฏิรูป...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6784,https://prachatai.com/print/37101,2011-09-27 21:43,คุยกับผู้ประสานงานเครือข่ายพลเมืองเน็ต: รัฐประ...,คุยกับอาทิตย์ สุริยะวงศ์กุล ผู้ประสานงานเครือข...,1,1,0,0,0,0,0,0,0,0,1,0,คุย อาทิตย์ สุริยะ วงศ์ กุล ผู้ประสานงาน เครือ...
6785,https://prachatai.com/print/7027,2006-01-18 04:55,ส.ว.แจ้งความดำเนินคดี คตง.ไม่คืนตำแหน่งจารุวรรณ,ประชาไท - 18 ต.ค.48 พล.ต.อินทรัตน์ ยอดบาง...,1,0,0,0,0,0,0,0,0,0,0,0,ประชา อิน รัตน์ ยอด เตย เชียงใหม่ ประธาน ค...
6786,https://prachatai.com/print/68204,2016-10-05 13:29,ประยุทธ์ปัดตอบปมกักตัว 'โจชัว หว่อง' นักเคลื่อ...,5 ต.ค. 2559 จากกรณี โจชัว หว่อง แกนนำนักเรียนน...,1,1,0,1,0,0,0,0,0,0,0,0,กรณี โจชัว หว่อง แกนนำ นักเรียน นักศึกษา ฮ่องก...
6787,https://prachatai.com/print/50808,2013-12-30 05:10,สมัครเลือกตั้ง ปัตตานี-ยะลา-นราธิวาส วันที่สอง...,'ซูการ์โน มะทา' นำผู้สมัคร ส.ส.เพื่อไทยลงครบ 3...,1,0,0,0,0,0,0,0,0,0,0,0,ผู้สมัคร ไทย เขต ยะลา ผู้สนับสนุน แห่ ให้กำลัง...


In [10]:
x = df['text_tokens']
y = df[df.columns[4:-1]].values

In [11]:
MAX_FEATURES = 20000
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                              output_sequence_length=1800,
                              output_mode='int')
# max vocab : 20000
# output len of vectorization : 1800 and pad

In [12]:
vectorizer.adapt(x.values) # create vocab

In [13]:
vectorized_text = vectorizer(x.values) # vectorize

In [14]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y)) # slice tensor to dataset

In [15]:
# list(dataset.as_numpy_iterator().next())

In [16]:
len(dataset)

6789

In [17]:
dataset = dataset.cache()
dataset = dataset.shuffle(6789)
dataset = dataset.batch(32)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

In [18]:
batch_x,  batch_y = dataset.as_numpy_iterator().next()

In [19]:
train = dataset.take(int(len(dataset)*.7)) # train 70%
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2)) # validation 20%
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1)) # test 10 %

In [20]:
train_generator = train.as_numpy_iterator()

In [21]:
# train_generator.next()

In [22]:
y.shape

(6789, 12)

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

model = Sequential()
# embedding : max_features was shifted for 1 and send 64 output with 3D
model.add(Embedding(MAX_FEATURES+1, 64))

# LSTM : required 3D input and send 3D output
model.add(Bidirectional(LSTM(64, return_sequences = True)))
model.add(Dropout(0.2))

# LSTM : required 3D input and send 2D output
model.add(Bidirectional(LSTM(32, activation = 'tanh')))
model.add(Dropout(0.3))

# hidden layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))

# output layer : 12 topics
model.add(Dense(12, activation='softmax'))

# define optimizer
optz = tf.keras.optimizers.Adam(lr = 0.001)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          1280064   
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        66048     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, None, 128)         0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 128)               8

C:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [24]:
model.compile(loss='BinaryCrossentropy', optimizer=optz, metrics = ['accuracy'])

In [25]:
history = model.fit(train, epochs=10, validation_data=val)

Epoch 1/10
149/149 [==============================] - 423s 3s/step - loss: 0.3615 - accuracy: 0.5795 - val_loss: 0.3165 - val_accuracy: 0.6198
Epoch 2/10
149/149 [==============================] - 317s 2s/step - loss: 0.3096 - accuracy: 0.6183 - val_loss: 0.2885 - val_accuracy: 0.6354
Epoch 3/10
149/149 [==============================] - 315s 2s/step - loss: 0.2925 - accuracy: 0.6403 - val_loss: 0.2726 - val_accuracy: 0.6481
Epoch 4/10
149/149 [==============================] - 311s 2s/step - loss: 0.2799 - accuracy: 0.6399 - val_loss: 0.2782 - val_accuracy: 0.6042
Epoch 5/10
149/149 [==============================] - 311s 2s/step - loss: 0.2662 - accuracy: 0.6556 - val_loss: 0.2465 - val_accuracy: 0.6503
Epoch 6/10
149/149 [==============================] - 303s 2s/step - loss: 0.2489 - accuracy: 0.6602 - val_loss: 0.2238 - val_accuracy: 0.7180
Epoch 7/10
149/149 [==============================] - 313s 2s/step - loss: 0.2275 - accuracy: 0.6789 - val_loss: 0.2061 - val_accuracy: 0.7247

In [26]:
# save  model
from tensorflow.keras.models import load_model
# model.save('data/news_topic_clf_model.h5')

In [45]:
model = load_model('data/news_topic_clf_model.h5')

In [46]:
batch_x, batch_y = test.as_numpy_iterator().next()

In [47]:
(model.predict(batch_x) > 0.5).astype(int)

1/1 [==============================] - 1s 1s/step


array([[0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0,

In [48]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

pre = Precision()
recall = Recall()
acc = CategoricalAccuracy()

In [49]:
for batch in test.as_numpy_iterator():
    # unpack batch
    x_true, y_true = batch
    # make precision
    y_hat = model.predict(x_true)
    
    y_true = y_true.flatten()
    y_hat = y_hat.flatten()
    
    pre.update_state(y_true, y_hat)
    recall.update_state(y_true, y_hat)
    acc.update_state(y_true, y_hat)

1/1 [==============================] - 0s 233ms/step


In [50]:
print(f'Precision : {pre.result().numpy()}\nRecall : {recall.result().numpy()}\nAccuracy : {acc.result().numpy()}')

Precision : 0.8703108429908752
Recall : 0.7388535141944885
Accuracy : 0.095238097012043


In [51]:
def ana_text(text):
    text = text_preprocess(text)
    vectorized_txt = vectorizer([text])
#     res = model.predict(np.expand_dims(vectorized_txt, 0))
    res = model.predict(vectorized_txt)
    
    txt = ''
    for idx, col in enumerate(df.columns[4:-1]):
        txt += '{}: {}\n'.format(col, res[0][idx]>0.5)
    
    return txt

In [52]:
txt = '''
ชาวอิรักจำนวนมากร่วมพิธีศพของผู้เสียชีวิตในเหตุเพลิงไหม้งานแต่งงานในเมืองการากอช ซึ่งทำให้มีผู้เคราะห์ร้ายมากกว่า 100 ราย ขณะตำรวจจับกุมผู้ต้องสงสัยที่อาจต้องรับผิดชอบต่อโศกนาฏกรรมครั้งนี้แล้ว 14 คน

สำนักข่าวต่างประเทศรายงานว่า ในช่วงบ่ายวันพุธที่ 27 ก.ย. 2566 ชาวอิรักจำนวนมากออกมาร่วมพิธีศพผู้เสียชีวิตมากกว่า 40 ศพ ที่สุสานในเมืองการากอช ในจังหวัดนิเนเวห์ ทางเหนือของประเทศ บ้างช่วยกันแห่โลงศพ ขณะที่บางคนถือรูปบุคคลอันเป็นที่รักผู้ล่วงลับมาร่วมพิธีด้วยความโศกสลด

โศกนาฏกรรมครั้งนี้เกิดขึ้นเมื่อเวลาประมาณ 22.00น. ถึง 22.45 น. วันอังคารที่ 26 ก.ย. ตามเวลาท้องถิ่น แขกจำนวน 1,000 ถึง 1,1000 คนไปร่วมฉลองพิธีแต่งงานของบ่าวสาวคู่หนึ่งที่โถงจัดงานเลี้ยงในเมืองการากอช ก่อนจะเกิดไฟไหม้ระหว่างที่บ่าวสาวเต้นรำและไฟลุกลามอย่างรวดเร็ว

เจ้าหน้าที่ความมั่นคงดำเนินการจับกุมผู้ที่อาจต้องรับผิดชอบต่อเหตุการณ์นี้แล้ว 14 คน ประกอบด้วย เจ้าของอาคาร, พนักงาน 10 คน และเจ้าหน้าที่ 3 คนที่เกี่ยวข้องกับการจัดการดอกไม้ไฟ หลังผลการตรวจสอบพบว่า อาคารจัดเลี้ยงแห่งนี้ไม่ได้มาตรฐานความปลอดภัย

ศูนย์อำนวยการป้องกันพลเรือนของอิรักระบุก่อนหน้านี้ว่า โถงจัดเลี้ยงแห่งนี้ถูกปูด้วยแผ่นฝ้าที่มีองค์ประกอบโลหะไวไฟสูง ซึ่งผิดกฎหมายในอิรัก และฝ้าดังกล่าวพังลงมาภายในไม่กี่นาทีหลังจากเกิดเพลิงไหม้ ฝ้าประเภทนี้ยังมีความเป็นพิษสูงเมื่อถูกเผาไหม้ด้วย

หลังเกิดเหตุ นายกรัฐมนตรี โมฮัมเหม็ด ชีอา อัล-ซูดานี ให้คำมั่นว่าจะตั้งคณะกรรมการสืบสวนเพื่อตรวจสอบอุบัติเหตุครั้งนี้โดยละเอียด เพื่อหาสาเหตุและดูว่ามีจุดใดที่ถูกละเลยหรือไม่ เขายังสั่งให้เจ้าหน้าที่ที่เกี่ยวข้อง เพิ่มความเข้มข้นในการตรวจสอบและยืนยันระบบความปลอดภัยของศูนย์การค้า, ร้านอาหาร, โถงจัดงาน และโรงแรมต่างๆ ด้วย'''

In [53]:
print(ana_text(txt))

1/1 [==============================] - 1s 1s/step
politics: True
human_rights: False
quality_of_life: False
international: True
social: False
environment: False
economics: False
culture: False
labor: False
national_security: False
ict: False
education: False

